In [ ]:
import json
from pathlib import Path

import torch
import torchaudio


In [ ]:
'''
Let's only use testset
'''
!wget http://download.magenta.tensorflow.org/datasets/nsynth/nsynth-test.jsonwav.tar.gz


In [ ]:
!tar -xf nsynth-test.jsonwav.tar.gz

In [ ]:
meta = json.load(open('nsynth-test/examples.json'))
len(meta)